In [ ]:
# ai서버 요청 코드

In [ ]:
import requests
from aws_requests_auth.aws_auth import AWSRequestsAuth
import json

# AWS 설정 (파일에서 읽기)
with open('../../../../keys/aws_accesskey.txt', 'r') as f:
    access_key = f.read().strip()

with open('../../../../keys/aws_secretkey.txt', 'r') as f:
    secret_key = f.read().strip()

# 기타 AWS 정보
region = 'ap-northeast-2'
service = 'sagemaker'
endpoint = 'runtime.sagemaker.ap-northeast-2.amazonaws.com'
url = 'https://runtime.sagemaker.ap-northeast-2.amazonaws.com/endpoints/dawoncecdEP/invocations'

# AWS Signature V4 인증 설정
auth = AWSRequestsAuth(
    aws_access_key=access_key,
    aws_secret_access_key=secret_key,
    aws_host=endpoint,
    aws_region=region,
    aws_service=service
)

# 요청 본문 (JSON)
data = {
    "text": "sample"
}

# POST 요청 보내기
response = requests.post(url, auth=auth, json=data)

# 응답 출력
print("Response Code:", response.status_code)
print("Response Body:", response.text)


In [ ]:
# 테스트 데이터 불러오는 코드

In [ ]:
def load_manual_data(file_paths):
    """
    주어진 파일 목록에서 <docsX/> 태그로 구분된 manual 데이터를 로드하고,
    각각을 리스트의 원소로 처리하는 함수.
    """
    manuals = []
    
    # 각 파일에서 데이터를 읽어와 <docsX/> 태그로 구분된 매뉴얼을 하나씩 처리
    for file_path in file_paths:
        current_manual = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                # <docsX/> 태그를 만나면 이전 매뉴얼을 저장하고 새로운 매뉴얼 시작
                if line.startswith("<docs"):
                    if current_manual:
                        # 이전 매뉴얼을 리스트에 추가
                        manuals.append('\n'.join(current_manual))
                        current_manual = []
                # 내용이 있는 줄은 현재 매뉴얼에 추가
                elif line:
                    current_manual.append(line)

        # 마지막 매뉴얼도 추가
        if current_manual:
            manuals.append('\n'.join(current_manual))

    # 샘플로 첫 3개의 manual을 출력
    for i in range(min(3, len(manuals))):
        print(f"Sample Manual {i+1}:")
        print(manuals[i][:200])  # 첫 200자만 출력
        print("=" * 50)

    return manuals

# manual1.txt와 manual2.txt에서 manual 데이터를 로드하고 통합
manual_files = ['../test_data/manual1.txt', '../test_data/manual2.txt']
manuals = load_manual_data(manual_files)


In [ ]:
manuals[0]

In [ ]:
def load_pair_data(file_path):
    """
    m과 d 쌍의 데이터를 불러오는 함수.
    매뉴얼 내용과 대화 내용을 간단한 구조로 저장하고,
    대화는 상황실과 신고자의 발화로 구분하여 리스트로 저장.
    """
    pairs = []
    current_manual_text = []
    current_dialogue_turns = []
    is_manual = True  # 매뉴얼과 대화를 구분하기 위한 플래그

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()

            # 매뉴얼 텍스트가 시작되면 is_manual 플래그를 True로 설정
            if line.startswith("<m"):
                if current_manual_text and current_dialogue_turns:  # 이전 매뉴얼 저장
                    pairs.append({
                        'manual': '\n'.join(current_manual_text),
                        'dialogue': current_dialogue_turns
                    })
                current_manual_text = []
                current_dialogue_turns = []
                is_manual = True
            # 대화 텍스트가 시작되면 is_manual 플래그를 False로 설정
            elif line.startswith("<d"):
                is_manual = False
            elif is_manual:
                # 매뉴얼 내용 추가
                current_manual_text.append(line)
            elif not is_manual and line:
                # 대화를 턴 별로 구분하여 저장
                if "상황실:" in line:
                    #current_dialogue_turns.append(line.replace("상황실:", "").strip())
                    current_dialogue_turns.append(line.strip())
                elif "신고자:" in line:
                    #current_dialogue_turns.append(line.replace("신고자:", "").strip())
                    current_dialogue_turns.append(line.strip())

    # 마지막 매뉴얼과 대화 저장
    if current_manual_text and current_dialogue_turns:
        pairs.append({
            'manual': '\n'.join(current_manual_text),
            'dialogue': current_dialogue_turns
        })

    # 샘플로 첫 3개의 m-d 쌍 출력
    for i, pair in enumerate(pairs[:3]):
        print(f"Sample Pair {i+1}:")
        print("Manual:")
        print(pair['manual'][:200])  # 매뉴얼 텍스트의 첫 200자 출력
        print("Dialogue:")
        print(pair['dialogue'][:6])  # 첫 6턴의 대화 출력
        print("=" * 50)

    return pairs

# manual_question_integrated.txt에서 m과 d 쌍 데이터를 로드
pairs = load_pair_data('../test_data/manual_question_integrated.txt')


In [ ]:
pairs

In [ ]:
def extend_pairs(pairs, start_percentage=0.4):
    """
    pairs에서 각 dialogue를 50% 지점까지 고정한 후, 두 문장씩 늘려가며 데이터를 확장하는 함수.
    """
    pairs_extended = []

    for pair in pairs:
        manual = pair['manual']
        dialogue = pair['dialogue']

        num_sentences = len(dialogue)
        start_index = int(num_sentences * start_percentage)  # 50% 지점

        # 50%까지의 고정된 부분 추가
        fixed_dialogue = dialogue[:start_index]
        pairs_extended.append({
            'manual': manual,
            'dialogue': fixed_dialogue  # 50% 고정 부분
        })

        # 50% 이후로 두 문장씩 확장
        for i in range(start_index, num_sentences, 2):
            extended_dialogue = dialogue[:i + 2]
            pairs_extended.append({
                'manual': manual,
                'dialogue': extended_dialogue
            })

    # 샘플로 첫 3개의 확장된 데이터를 출력
    for i, pair in enumerate(pairs_extended[:3]):
        print(f"Extended Pair {i+1}:")
        print("Manual:")
        print(pair['manual'][:200])  # 첫 200자만 출력
        print("Dialogue:")
        print(pair['dialogue'])  # 대화 문장 전체 출력
        print("=" * 50)

    return pairs_extended

# pairs 확장
pairs_extended = extend_pairs(pairs)


In [ ]:
len(pairs_extended)

In [ ]:
# 테스팅 코드 시작